In [1]:
from pyarrow.parquet import ParquetFile

ParquetFile('../Data/kaggle-belka-shrunken/train.parquet').metadata

  created_by: parquet-cpp-arrow version 15.0.2
  num_columns: 7
  num_rows: 98415610
  num_row_groups: 94
  format_version: 2.6
  serialized_size: 86163

In [2]:
import pandas as pd
import pyarrow.dataset as ds

n = 1000
path = "../Data/kaggle-belka-shrunken/train.parquet"
df = ds.dataset(path).scanner().head(n).to_pandas()
df
#training_data = pd.read_parquet('../Data/kaggle-belka-shrunken/train.parquet', engine='pyarrow', nrows = 1000)

,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,binds_BRD4,binds_HSA,binds_sEH
0,0,0,0,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,0,0,0
1,0,0,1,C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...,0,0,0
2,0,0,2,C#CCOc1ccc(CNc2nc(NCc3ccc(OCC#C)cc3)nc(N[C@@H]...,0,0,0
3,0,0,6,C#CCOc1ccc(CNc2nc(NCCNC(=O)C(=C)C)nc(N[C@@H](C...,0,0,0
4,0,0,10,C#CCOc1ccc(CNc2nc(NCC(=O)NCC=C)nc(N[C@@H](CC#C...,0,0,0
...,...,...,...,...,...,...,...
995,0,1,657,C#CCOc1cccc(CNc2nc(NCc3cnn(Cc4ccccc4)c3)nc(N[C...,0,0,0
996,0,1,658,C#CCOc1cccc(CNc2nc(NCc3cnns3)nc(N[C@@H](CC#C)C...,0,0,0
997,0,1,659,C#CCOc1cccc(CNc2nc(NCc3nc(-c4ccsc4)no3)nc(N[C@...,0,0,0
998,0,1,660,C#CCOc1cccc(CNc2nc(NCc3nc4c(s3)CCCC4)nc(N[C@@H...,0,0,0


In [9]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np

#initialise seed
np.random.seed(69)
#load data
path = "../Data/kaggle-belka-shrunken/train.parquet"
training_data = pq.ParquetFile(path)
#get total rows from training data
row_num = training_data.metadata.num_rows
#determine sample size (w/o replacement)
sample_size = 1000000
#obtain 1-3 million random indicies 
sample_indices = np.random.choice(row_num, size=sample_size, replace=False)
sample_indices_set = set(sample_indices)

#initialise empty list for subsetted data and counter for index
random_subset_data = []
current_row_index = 0

#Iterate through num_row_groups. the entire df is split into 94 row-length groups. this code should work
#for original train.parquet which has 282 groups too. 
for row_group_index in range(training_data.metadata.num_row_groups):
    #subsets df based on row groups, convert to pandas
    table = training_data.read_row_group(row_group_index)
    df = table.to_pandas()
    row_group_size = len(df)
    #obtain range of indices in subsetted table
    row_group_indices = range(current_row_index, current_row_index + row_group_size)
    #extract indicies intersecting with our random sampled index list
    intersecting_indices = sample_indices_set.intersection(row_group_indices)

    if intersecting_indices:
        #correct indicies of intersecting indicies to the index in table
        adjusted_indices = [index - current_row_index for index in intersecting_indices]
        selected_rows = df.iloc[adjusted_indices]
        #append rows to temp storage variable
        random_subset_data.append(selected_rows)

    #update current row index
    current_row_index += row_group_size
    
    #break loop once desired number is reached
    if len(random_subset_data) >= sample_size:
        break
#store within actual df
random_subset_df = pd.concat(random_subset_data, ignore_index=False)
len(random_subset_df)

#53s run time


1000000

In [10]:
random_subset_df

,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,binds_BRD4,binds_HSA,binds_sEH
98309,0,185,494,C#CC[C@@H](CC(=O)N[Dy])Nc1nc(NCc2ccc(CN3C(=O)C...,0,0,0
131078,0,251,78,C#CC[C@@H](CC(=O)N[Dy])Nc1nc(Nc2ccc(C)cc2CO)nc...,0,0,0
884744,2,303,789,C#CC[C@@](C)(Nc1nc(NCCC(=O)Nc2ccncc2)nc(Nc2ccs...,0,0,0
294924,0,564,327,C#CC[C@@H](CC(=O)N[Dy])Nc1nc(NCc2sccc2C)nc(Nc2...,0,0,0
294929,0,564,334,C#CC[C@@H](CC(=O)N[Dy])Nc1nc(NCc2cc(C)cnc2F)nc...,0,0,0
...,...,...,...,...,...,...,...
589801,270,103,760,CN1CCO[C@@H](CNc2nc(Nc3ccc4nccn4c3)nc(N[C@@H](...,0,0,0
851954,270,602,575,[N-]=[N+]=NCCC[C@H](Nc1nc(NCC2(O)CCSC2)nc(Nc2c...,0,0,0
753660,270,421,32,CC(C)CC(C)(CNc1nc(NCc2cnc3n2CCOC3)nc(N[C@@H](C...,0,0,0
786429,270,483,625,[N-]=[N+]=NCCC[C@H](Nc1nc(NCc2ccc(C(=O)N3CCCC3...,0,0,0
